<a href="https://colab.research.google.com/github/tak-creator/giganci/blob/main/Wczytywanie_danych_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#układ folderu dla pracy domowej
https://drive.google.com/file/d/1Y_d8DQ0lEBYco5kX6BFCS5jt7Rg0Djfp/view?usp=drive_link

#import niezbędnych bibliotek
import os
from sklearn.utils import shuffle
import cv2 as cv
import numpy as np
import tensorflow as tf


#wczytywanie danych wraz z etykietami
#lista klas które mamy do klasyfikacji (nazw pod-podfolderów)

class_names = ['0', '1']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
IMAGE_SIZE = (128,128)

#załadowanie grafik i dodanie ich do odpowiednich list - ten krok
#można wysłać uczniom jako gotowiec

def load_data():
    #DIRECTORY - nazwa folderu głównego, CATEGORY podfoldery uczące i testowy
    DIRECTORY = "learning"
    CATEGORY =["train", "test"]
    output = []

    for category in CATEGORY:
        path = os.path.join(DIRECTORY,category)
        images =[]
        labels = []

        for folder in os.listdir(path):
             label = class_names_label[folder]

             for file in os.listdir(os.path.join(path,folder)):
                 img_path = os.path.join(os.path.join(path,folder), file)
                 image = cv.imread(img_path)
                 image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
                 image = cv.resize(image,IMAGE_SIZE)

                 images.append(image)
                 labels.append(label)
        images = np.array(images, dtype = "float32")
        labels = np.array(labels,dtype="int32")

        output.append((images,labels))
    return output


(train_images, train_labels), (test_images,test_labels) = load_data()

train_images,train_labels = shuffle(train_images,train_labels,random_state=25)

In [ ]:
#stworzenie modelu prostą siecią neuronową
model = tf.keras.models.Sequential([
   tf.keras.layers.Rescaling(1./255, input_shape=(128, 128)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(2,activation = tf.nn.softmax)
])

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),  # Nowa warstwa
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),# Nowa warstwa
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
#trening modelu
model.compile(optimizer = "adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics =["accuracy"])
history = model.fit(train_images,train_labels,batch_size = 3, epochs = 100, validation_split = 0.2)

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

#sprawdzenie poprawności działania modelu
for i in range(len(test_images)):
    predicted = np.argmax(predictions[i])
    label = test_labels[i]
    print(f"Recgonized {predicted} on class {label}")

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.3726 - loss: 0.7278 - val_accuracy: 0.5000 - val_loss: 0.6615
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4813 - loss: 1.1580 - val_accuracy: 0.5000 - val_loss: 0.6502
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5008 - loss: 1.1112 - val_accuracy: 0.7500 - val_loss: 0.6015
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7452 - loss: 0.8349 - val_accuracy: 0.7500 - val_loss: 0.6034
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5778 - loss: 0.8609 - val_accuracy: 0.7500 - val_loss: 0.5878
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6210 - loss: 0.6928 - val_accuracy: 0.7500 - val_loss: 0.5610
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3583 - loss: 0.8218 - val_accuracy: 0.7500 - val_loss: 0.5552
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6583 - loss: 1.1163 - val_accuracy: 0.7500 - val_loss: